In [1]:
# Adapted from “Creating Blackjack in Python.” YouTube, https://www.youtube.com/watch?v=C82s5WufNUA. 
class Card: 
    def __init__(self , value, suit): 
        self.cost = value 
        self.value = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K'] [value-1]
        self.suit = ['♠️','♦','♣','♥'][suit]

        
    def price(self):         # get value of non-numeric cards 
        if self.cost >= 10: 
            return 10 
        elif self.cost == 1: 
            return 11 
        return self.cost 
    
    def show_card(self): 
        print('\u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510')
        print(f'\u2502{self. value: <2}     \u2502')
        print('\u2502       \u2502 ') 
        print(f'\u2502   {self.suit}   \u2502')
        print('\u2502       \u2502 ')
        print(f'\u2502     {self. value:>2}\u2502')
        print('\u2514\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2518')
        #print('\u2660\uFE0F')
        #print('\u2666')
        #print('\u2663')
        #print('\u2665')
        #U+2660	U+2665	U+2666	U+2663
        

In [2]:
import random 
# from card import Card

class Deck: 
    def __init__(self): 
        self.cards = []
        self.used_cards = []  # store used cards
        
    def generate(self): 
        for i in range(1, 14): 
            for j in range(4):
                self.cards.append(Card(i, j))  # generate deck with different suits

    def draw(self, amount):
        cards = [] 
        for i in range(amount): 
            card = random.choice(self.cards)
            self.cards.remove(card)
            self.used_cards.append(card)  # add the drawn card to used cards list
            cards.append(card)
        return cards 

    def reshuffle(self):
        self.cards.extend(self.used_cards)  # put all used cards back into the deck
        self.used_cards = []        # clear the used cards list
        random.shuffle(self.cards)  # shuffle the deck

In [3]:
# from deck import Deck 

class Player: 
    def __init__(self, isDealer, deck):
        self.cards = []
        self.selected_cards = []  # store selected card values
        self.selected_suits = []  # store selected suits
        self.isDealer = isDealer 
        self.deck = deck 
        self.score = 0 
        
    def reset(self):               # method to continue games as new
        self.cards = []
        self.selected_cards = []  # store selected card values
        self.selected_suits = []  # store selected suits
        self.score = 0
        
    def hit(self): 
        selected = self.deck.draw(1) # draw one card
        self.cards.extend(selected)
        self.selected_cards.extend([card.value for card in selected])  # get selected card value
        self.selected_suits.extend([card.suit for card in selected])    # get selected suit
        self.check_score()
        if self.score > 21: 
            return 1
        if self.score == 21:
            return 2
        return 0 
    
    def deal(self): 
        selected = self.deck.draw(2) # draw two cards
        self.cards.extend(selected)
        self.selected_cards.extend([card.value for card in selected])  # get selected card value
        self.selected_suits.extend([card.suit for card in selected])    # get selected suit
        self.check_score()
        if self.score == 21:
            return 1 
        return 0 
    
    def check_score(self): 
        ace_counter = 0 
        self.score = 0 
        for card in self.cards:
            if card.price() == 11:
                ace_counter += 1 
            self.score += card.price()
        while ace_counter != 0 and self.score > 21: 
            ace_counter -= 1 
            self.score -= 10 
        return self.score 
    
    def show(self): 
        if self.isDealer == True:
            print("Dealer's Cards") 
        else: 
            print("Player's Cards")
            
        for i in self.cards:
            i.show_card() 
            
        print("Score: " + str(self.score))
        print("Selected Cards:", self.selected_cards) # showing current hand 
        print("Selected Suits:", self.selected_suits)


In [4]:
# from deck import Deck 
# from player import Player 

class Blackjack: 
    def __init__(self): 
        self.deck = Deck() 
        self.deck.generate() 
        self.player = Player(False, self.deck)
        self.dealer = Player(True, self.deck)
        
    def play(self):
        continue_game = "yes"
        while continue_game.lower() == "yes":           # keep the session going
            self.player.reset()    # reset player and dealer instances for each game
            self.dealer.reset()
            
            p_status = self.player.deal() 
            d_status = self.dealer.deal() 

            self.player.show()

            if p_status == 1: 
                self.dealer.show()                    # show dealers cards after to check for tie
                if d_status == 1: 
                    print("Dealer and Player got Blackjack! It's a push. (Tie)") # Blackjack on first draw for both
                else:
                    print("Player got Blackjack! Congrats!") # Blackjack on first draw
            else:

                cmd = "" 
                while cmd != "Stand":
                    bust = 0
                    cmd = input("Hit or Stand? ") 
                    if cmd.lower() == "hit" and len(self.deck.cards) != 0: 
                        bust = self.player.hit() 
                        self.player.show() 
                    if bust == 1:
                        print("Player busted. Good Game!")
                        break
                    if bust == 2: # forces stand for dealer to play once
                        break
                    if len(self.deck.cards) == 0: # breaks out of loop if out of cards
                        break
                print("\n") 
                self.dealer.show() 
                if d_status == 1 and bust != 1:
                    print("Dealer got Blackjack! Better luck next time!") # Blackjack on first draw

                elif bust != 1 and bust != 2: # if player busts or gets 21 do not let dealer play
                    while self.dealer.check_score() < 17 and len(self.deck.cards) != 0:
                        if self.dealer.hit() == 1:
                            self.dealer.show()
                            print("Dealer busted. Congrats!") 
                            break
                        self.dealer.show()

                    if len(self.deck.cards) == 0: 
                        print("Out of cards! Need to reshuffle.")
                        self.deck.reshuffle()
                        
                    if self.dealer.check_score() <= 21: # if dealer scores 17-21 they have a chance of winning
                        if self.dealer.check_score() == self.player.check_score(): # if dealer scores 17-21 they win if > player
                            print("It's a Push (Tie). Better luck next time!")
                        elif self.dealer.check_score() > self.player.check_score():
                            print("Dealer wins. Good Game!")
                        elif self.player.check_score() > self.dealer.check_score(): #if player > dealer and < 21 they win
                            print("Player wins. Congratulations!")
                elif bust == 2:
                    print("Player wins. Congratulations!")
                
            if len(self.deck.cards) <= 4: 
                print("\n")
                print("Out of cards! Need to reshuffle.")
                self.deck.reshuffle()
                
            continue_game = input("Do you want to continue? (yes/no): ")
            print("\n")
                
b = Blackjack()
b.play()


Player's Cards
┌───────┐
│Q      │
│       │ 
│   ♦   │
│       │ 
│      Q│
└───────┘
┌───────┐
│7      │
│       │ 
│   ♥   │
│       │ 
│      7│
└───────┘
Score: 17
Selected Cards: ['Q', '7']
Selected Suits: ['♦', '♥']
Hit or Stand? Stand


Dealer's Cards
┌───────┐
│8      │
│       │ 
│   ♠️   │
│       │ 
│      8│
└───────┘
┌───────┐
│A      │
│       │ 
│   ♥   │
│       │ 
│      A│
└───────┘
Score: 19
Selected Cards: ['8', 'A']
Selected Suits: ['♠️', '♥']
Dealer wins. Good Game!
Do you want to continue? (yes/no): yes


Player's Cards
┌───────┐
│J      │
│       │ 
│   ♦   │
│       │ 
│      J│
└───────┘
┌───────┐
│9      │
│       │ 
│   ♣   │
│       │ 
│      9│
└───────┘
Score: 19
Selected Cards: ['J', '9']
Selected Suits: ['♦', '♣']
Hit or Stand? Stand


Dealer's Cards
┌───────┐
│2      │
│       │ 
│   ♥   │
│       │ 
│      2│
└───────┘
┌───────┐
│J      │
│       │ 
│   ♣   │
│       │ 
│      J│
└───────┘
Score: 12
Selected Cards: ['2', 'J']
Selected Suits: ['♥', '♣

### Bug fixes

In [5]:
# fixed skipping heart suit bug
# fixed game stopping if dealer score was less than player score
# fixed dealer always busting
# player stands on 21, allows dealer to go for Blackjack
# neither player nor dealer can recieve win messsage after bust
# cards may only be played once unless the deck is reshuffled
# deck persists through games

### Changes needed

In [6]:
# win immediately when player lands exaclty on 21? - dealer allowed to show cards to tie

In [7]:
#Train AI(neural network) to replace a player and play against the dealer.